# Evaluate the performance of score methods

This script evaluates the performance of score methods(SAPS-II, modified SAPS-II and SOFA) on mortality prediction tasks.

In [1]:
from __future__ import print_function

import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
from multiprocessing import Pool, cpu_count
import re
import traceback
import random
import shutil

from utils import getConnection
from utils import parseUnitsMap
from utils import parseNum
from utils import sparsify

%matplotlib inline

## Calculate metrics of performance

scorefunc is the metrics, we can switch it between AUROC(roc_auc_score) and AUPRC(average_precision_score).

In [3]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score, roc_curve, average_precision_score
from scipy import stats

# scorefunc = roc_auc_score
scorefunc = average_precision_score

def get_auc_score(X, y, labelname, labeln, y_pred=None, model=None, foldpath=None, scorefunc=scorefunc):
    aucs = []
    foldsres = np.load(foldpath)[labelname][labeln][0]
    for train, valid, test in foldsres:
        train = np.concatenate((train, valid))
        Xtrain, Xtest = X[train,:], X[test,:]
        ytrain, ytest = y[train], y[test]
        if y_pred is not None:
            y_pred_test = y_pred[test]
            aucs.append(scorefunc(ytest, y_pred_test))
        elif model is not None:
            model.fit(Xtrain, ytrain)
            y_pred_test = model.predict_proba(Xtest)[:,1]
            aucs.append(scorefunc(ytest, y_pred_test))
#         print(roc_curve(ytest, y_pred_test))
    aucmean = np.mean(aucs)
    aucstd = np.std(aucs)
    dist = 1.96 * aucstd
    return aucmean, aucstd, [aucmean-dist, aucmean+dist] # http://biostats.bepress.com/cgi/viewcontent.cgi?article=1307&context=ucbbiostat

## Print the metrics of performance of methods

We print the metrics of performance of 3 methods mentioned above, including mean, standard error and credible interval.

In [5]:
HRS = 24
WD = '../../Data/admdata_17f/%dhrs/non_series' % HRS
foldpath = '../../Data/admdata_17f/%dhrs/series/5-folds.npz' % HRS
foldcvpath = '../../Data/admdata_17f/%dhrs/series/cv/5-folds.npz' % HRS

mv_flag = np.array(np.load(os.path.join(WD, 'mv_flag.npy')).tolist())
sapsii = np.array(np.load(os.path.join(WD, 'sapsii.npz'))['sapsii'])
sofax = np.load(os.path.join(WD, 'sofa.npz'))['sofa'].reshape(-1, 1)
yall = np.genfromtxt(os.path.join(WD, 'output.csv'), delimiter=',')
sapsii_pred = sapsii[:, 4]
sapsii_score = sapsii[:, 5:].astype(np.float64)
sapsii_score[np.isnan(sapsii_score)] = 0.0

from sklearn.linear_model import LogisticRegression

for mp in [0,2,3,4,5]:
    y = yall[:, mp]
    sofam = LogisticRegression()
    print(mp)
    # cv sapsii
    t1 = datetime.datetime.now()
    cv_sapsii_res = get_auc_score(sapsii_score[~mv_flag], y[~mv_flag], labelname='folds_ep_mor', labeln=mp, y_pred=sapsii_pred[~mv_flag], model=None, foldpath=foldcvpath)
    t2 = datetime.datetime.now()
    print(t2-t1)
    # cv sapsii refitted
    cv_sapsii_refitted_res = get_auc_score(sapsii_score[~mv_flag], y[~mv_flag], labelname='folds_ep_mor', labeln=mp, y_pred=None, model=LogisticRegression(), foldpath=foldcvpath)
    # cv sofa
    cv_sofa_res = get_auc_score(sofax[~mv_flag].reshape(-1,1), y[~mv_flag], labelname='folds_ep_mor', labeln=mp, y_pred=None, model=LogisticRegression(), foldpath=foldcvpath)
    # # mv sapsii
    # print('mv sapsii:', get_auc_score(sapsii_score[mv_flag], y[mv_flag], 10, y_pred=sapsii_pred[mv_flag], model=None))
    # # mv sofa
    # print('mv sofa:', get_auc_score(sofax[mv_flag].reshape(-1,1), y[mv_flag], 10, y_pred=None, model=LogisticRegression()))
    # # mv sapsii refitted
    # print('mv sapsii refitted:', get_auc_score(sapsii_score[mv_flag], y[mv_flag], 10, y_pred=None, model=LogisticRegression()))
    # all sapsii
    all_sapsii_res = get_auc_score(sapsii_score, y, labelname='folds_ep_mor', labeln=mp, y_pred=sapsii_pred, model=None, foldpath=foldpath)
    # all sapsii refitted
    all_sapsii_refitted_res = get_auc_score(sapsii_score, y, labelname='folds_ep_mor', labeln=mp, y_pred=None, model=LogisticRegression(), foldpath=foldpath)
    # all sofa
    all_sofa_res = get_auc_score(sofax.reshape(-1,1), y, labelname='folds_ep_mor', labeln=mp, y_pred=None, model=LogisticRegression(), foldpath=foldpath)
    for t in [all_sapsii_res, all_sapsii_refitted_res, all_sofa_res, cv_sapsii_res, cv_sapsii_refitted_res, cv_sofa_res]:
        print(t[0], t[1], t[2])
    print('\n\n')

0
0:00:00.699072
0.358635826925 0.0051616056917 [0.34851907976964519, 0.36875257408110879]
0.398930603806 0.0120224811145 [0.37536654082121784, 0.42249466678991826]
0.319120543843 0.00851908762023 [0.3024231321071208, 0.33581795557842853]
0.362481065185 0.00652850750031 [0.34968519048422431, 0.3752769398854382]
0.403664261923 0.00690255915632 [0.39013524597622989, 0.41719327786899635]
0.327331566464 0.00674421464022 [0.3141129057696509, 0.34055022715932315]



2
0:00:00.050566
0.136097586693 0.0153290837308 [0.10605258258082959, 0.16614259080570057]
0.158718017566 0.0225631317659 [0.11449427930446827, 0.20294175582695378]
0.102733740209 0.0277561810112 [0.048331625426594103, 0.15713585499041538]
0.119047668948 0.0216778655276 [0.07655905251395545, 0.1615362853820605]
0.136780433646 0.0281294438591 [0.081646723681773406, 0.19191414360941622]
0.0881368475984 0.00866452086003 [0.071154386712732554, 0.1051193084840697]



3
0:00:00.034013
0.172997666265 0.0213739070552 [0.13110480843675182

In [4]:
print(sapsii)

[[58526 100001 275225 ..., 0 0 6]
 [9895 100006 291788 ..., 0 10 6]
 [23018 100007 217937 ..., 0 0 8]
 ..., 
 [19412 199995 255092 ..., 0 0 8]
 [27200 199998 293589 ..., 0 0 8]
 [40370 199999 211153 ..., 0 0 6]]
